## This noteboook reads raw annotation data from FlyBase, DGRP or DEST2 to transform it into dataframes.
- And add all the necessary information for each mutation. 

In [1]:
import pandas as pd 
import numpy as np
import re 
import os
from utils import alph
import scipy.stats as ss
from Bio import SeqIO
import statistics
import time

## DATA

In [ ]:
WORK_DIR = os.path.dirname(os.path.abspath('')) + os.sep


'''## DEST2
df_DEST2Raw  = pd.read_csv(f'{WORK_DIR}/data/DEST_all_snp', sep='\t')
## DGRP
df_DGRPRaw = pd.read_csv(f'{WORK_DIR}/data/dgrp.fb557.annot.txt', sep='\\t', names=['ID', 'allele', 'GeneAnnotation', 'RegulationAnnotation'])
## Lethal Mutations annotated on FlyBase
df_LethalRaw = pd.read_csv(f'{WORK_DIR}/data/LethalAllele_EMS.txt', sep='\t')
## Hypomorohic alleles retreived from FlyBase
df_HypomorphicRaw = pd.read_csv(f'../data/HypomorphicAllele_EMS.txt', sep='\t')  '''

## csv files
dc_proten_seqs = SeqIO.to_dict(SeqIO.parse(f'{WORK_DIR}/data/'+'dmel-all-translation-r6.44.fasta', "fasta")) 
df_Dmel_recap = pd.read_csv(f'{WORK_DIR}/csv/Dmel6.44PredictionsRecap.csv', sep=',', index_col=0)
df_Dmel_recap_pp = df_Dmel_recap.reset_index()[['FBpp_ID', 'FBtr_ID','FBgn_ID', 'GEMME_GlobalConfidence', 'F_obs', 'Length']]
df_mapping = pd.read_csv(f'{WORK_DIR}/csv/mapping_database.csv', index_col=0, sep=',')

## Different re patterns definition

In [7]:
## RE PATTERNS # Define regular expressions for pattern matching

patternPhenotype = re.compile(r'==\s*([^<]+?)\s*</cvterm>')
patternMutProt = re.compile('[A-Z]{1}[0-9]{1,5}[A-Z]{1} \| [0-9,A-Z,a-z,\-,\(,\)]*')

##DGRP
patternGene = re.compile('FBgn[0-9]*')
patternTrans = re.compile('FBtr[0-9]*')

patternMut = re.compile('[A-Z]{1}[0-9]{1,5}[A-Z]{1}')


## regular expressions
patternMutLethal = re.compile('(?<=Amino acid replacement: )[A-Z]{1}[0-9]{1,5}[A-Z]{1}')#[\.]
patternNAchange = re.compile('(?<= na_change=)[A-Z]{1,3}[0-9]{1,9}[A-Z]{1,3}')
termMut_pattern = re.compile('[A-Z]{1}[0-9]{1,5}term')
patternPhenotype = re.compile('(?<=\<newline\>)*all die *')

-  some proteins don't have the same annotanion/id between the sources 

In [3]:
# Mapping of three-letter to one-letter amino acid codes
aa_mapping = {
    'Ala': 'A', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D', 'Cys': 'C',
    'Glu': 'E', 'Gln': 'Q', 'Gly': 'G', 'His': 'H', 'Ile': 'I',
    'Leu': 'L', 'Lys': 'K', 'Met': 'M', 'Phe': 'F', 'Pro': 'P',
    'Ser': 'S', 'Thr': 'T', 'Trp': 'W', 'Tyr': 'Y', 'Val': 'V'
}

# Function to transform amino acid change notation
def transform_aa_change(aa_change):
    if aa_change.startswith('p.'):
        aa_change = aa_change[2:]  # Remove 'p.' prefix
        for three_letter, one_letter in aa_mapping.items():
            aa_change = aa_change.replace(three_letter, one_letter)
    return aa_change

def check_wt_aa(row):
    fbpp_id = row['FBpp_ID']
    aa_pos = row['Residue'] - 1  # Convert to 0-based index
    wt_aa = row['Mutation'][0]  # Wild-type amino acid
    seq_record = dc_proten_seqs.get(fbpp_id)
    if seq_record:
        return seq_record.seq[aa_pos] == wt_aa
    return False

# Create a dico with ProteoCast data for a specifi set of FBpp_ID
def create_gemme_dico(df):
    gemme_data = {}
    for fbpp_id in df['FBpp_ID'].unique():
        id = df_mapping.loc[fbpp_id, 'id']
        try:
            gemme_data[fbpp_id] = pd.read_csv(f'../Drosophila_ProteoCast/{id}/4.{fbpp_id}_ProteoCast.csv')
        except: 
            continue
    return gemme_data

# Function to get GEMME score and class for a given mutation
def get_gemme_score_class(row, gemme_data):
    fbpp_id = row['FBpp_ID']
    aa_change = row['Mutation']
    if fbpp_id in gemme_data:
        match = gemme_data[fbpp_id][gemme_data[fbpp_id]['Mutation'] == aa_change]
        if not match.empty:
            return match.iloc[0]['GEMME_score'], match.iloc[0].get('GEMME_rank', None), match.iloc[0]['Variant_class'], match.iloc[0].get('GEMME_LocalConfidence', None)
    return None, None, None, None

# Add zscore and class for the new values
def calculate_gmm3_class_and_zscore(row, gemme_data):
    FBpp = row['FBpp_ID']
    GEMME_score = row['GEMME_score']
    try:
        threshUnc = df_Dmel_recap.loc[FBpp, 'GMM3_uncertain']
    except:
        print('No threshold found for ', FBpp)
    G_N = gemme_data[FBpp].loc[gemme_data[FBpp]['GEMME_score'] >= threshUnc, 'GEMME_score'].tolist()
    if G_N:
        muN = statistics.mean(G_N)
        stdN = statistics.stdev(G_N)
        zscore = (GEMME_score - muN) / stdN
    else:
        zscore = np.nan
    
    return zscore


In [17]:
def CorrectProtname(prot):
    """if 'Cg25C' in prot or 'Hsp60' in prot:
        continue"""
    if 'eIF-4a' in prot:
        prot='eIF4A'+prot[-3:]
    if 'Cctgamma' in prot:
        prot='CCT3'+prot[-3:]
    if 'Itp-r83A' in prot:
        prot='Itpr'+prot[-3:]
    if 'RpII215-PA' in prot:
        prot='Polr2A-PA'
    if prot=='DNApol-gamma35-PA':
        prot='PolG2-PA'
    if 'Elf' in prot:
        prot='eRF3'+prot[-3:]
    if 'Argk' in prot:
        prot='Argk1'+prot[-3:]
    if 'CG15896' in prot:
        prot='mldr'+prot[-3:]
    if 'eIF-1A' in prot:
        prot='eIF1A'+prot[-3:]
    if 'RpII140' in prot:
        prot="Polr2B"+prot[-3:]
    if 'CG17829' in prot:
        prot='Hinfp'+prot[-3:]
    if 'nct' in prot:
        prot='Nct'+prot[-3:]
    if 'ade3' in prot:
        prot='Grat'+prot[-3:]
    if 'CG8184' in prot:
        prot="HUWE1"+prot[-3:]
    if 'Cg25C' in prot:
        prot='Col4a1'+prot[-3:]
    if 'Grat' in prot:
        prot='Gart'+prot[-3:]
    if prot=='tam-PA':
        prot='PolG1-PA'
    if 'Hsp60' in prot:
        prot='Hsp60A'+prot[-3:]
    if 'DNApol-alpha60-PA' in prot:
        prot='Prim2-PA'
    if 'mTor' in prot:
        prot='Tor'+prot[-3:]
    if 'Delta' in prot:
        prot='Dl'+prot[-3:]
    if 'Cs1' in prot:
        prot='kdn'+prot[-3:]
    if 'hry' in prot:
        prot='h'+prot[-3:]
    if 'nanos' in prot:
        prot='nos'+prot[-3:]
    return prot

## DEST2  
- file *dest.all.PoolSNP.001.50.24Aug2024.ann.vcf.gz* without population data

- "the current release includes 530 high quality Pool-Seq samples (>32,000 flies), comprising a combination of the previous DEST release with newly sequenced pools, collected between 2016 and 2021 by DrosEU, as well as publicly available Pool-Seq samples from published studies of wild-derived D. melanogaster"; 753 columns in the original file
- 4 801 078 lines #SNPs
- All kinds of SNPs 27 {'splice_region_variant&intron_variant', 'initiator_codon_variant', '3_prime_UTR_variant', 'intergenic_region', 'splice_donor_variant&intron_variant', 'splice_region_variant&stop_retained_variant', 'missense_variant', 'splice_acceptor_variant&intron_variant', 'downstream_gene_variant', 'initiator_codon_variant&splice_region_variant', 'synonymous_variant', 'intron_variant', 'upstream_gene_variant', 'splice_region_variant', 'stop_lost', '5_prime_UTR_premature_start_codon_gain_variant', 'splice_region_variant&synonymous_variant', 'start_lost&splice_region_variant', 'start_lost', 'stop_gained&splice_region_variant', 'stop_retained_variant', 'stop_gained', '5_prime_UTR_variant', 'stop_lost&splice_region_variant', 'splice_region_variant&non_coding_transcript_exon_variant', 'non_coding_transcript_exon_variant', 'missense_variant&splice_region_variant'}
- we are interested in missense_variant (and missense_variant&splice_region_variant? )
- 343 636 / 346 319 missense SNPs in at least one gene 
- among them there are mutations in pseudogene that are not included in 6.44 proteome (protein-coding but no known proteins... FBgn0031208)

In [ ]:
## retreiving all the SNP types associated to the SNPs
df_DEST2Raw['SNP_TYPE'] = df_DEST2Raw['INFO'].apply(lambda x: set(y.split('|')[1] for y in x.split(';')[5].split(',')))
snp_type_union = set.union(*df_DEST2Raw['SNP_TYPE'])
print(f'In total there are {len(snp_type_union)} SNPs types in the dataset')
## filter the SNPs labeled as missense variant
df_DEST_missense = df_DEST2Raw[df_DEST2Raw['SNP_TYPE'].apply(lambda x: 'missense_variant' in x)]
df_DEST_missense.reset_index(drop=True, inplace=True)
print(f'In total there are {len(df_DEST_missense)} missense SNPs in the dataset')
del df_DEST2Raw

In total there are 27 SNPs types in the dataset
In total there are 343636 missense SNPs in the dataset


In [16]:
del df_DEST2Raw

In [7]:
# Function to extract missense_variant annotations with additional information
def extract_missense_variants(info):
    
    annotations = info.split('ANN=')[1].split(',')
    ## example of an annotation 'C|missense_variant|MODERATE|CG11023|FBgn0031208|transcript|FBtr0300690|protein_coding|1/3|c.32A>C|p.Glu11Ala|183/1802|32/1443|11/480||'

    fbgn_list = []; fbtr_list = []; ## FBgn and FBtr IDs
    cdna_pos_list = [] ## cDNA position e.g c.32A>C
    aa_change_list = [] ##'p.Glu11Ala'
    aa_pos_list = []; protein_length_list = []
    for annotation in annotations:
        fields = annotation.split('|')
        if fields[1] == 'missense_variant':
            fbgn_list.append(fields[4])
            fbtr_list.append(fields[6])
            cdna_pos_list.append(fields[9])
            aa_change = transform_aa_change(fields[10])
            aa_change_list.append(aa_change)
            aa_pos, protein_length = fields[13].split('/')
            aa_pos_list.append(int(aa_pos))
            protein_length_list.append(int(protein_length))
    return fbgn_list, fbtr_list, cdna_pos_list, aa_change_list, aa_pos_list, protein_length_list

In [ ]:

# Apply the function to the DataFrame
df_DEST_missense[['FBgn_ID', 'FBtr_ID', 'cDNA_pos', 'Mutation', 'Residue', 'Length']] = df_DEST_missense['INFO'].apply(lambda x: pd.Series(extract_missense_variants(x)))
# Explode the lists in the DataFrame
df_DEST_missense = df_DEST_missense.explode(['FBgn_ID', 'FBtr_ID', 'cDNA_pos', 'Mutation', 'Residue', 'Length'])
# Reset the index after exploding
df_DEST_missense.reset_index(drop=True, inplace=True)

# Filter snp_Dest_Missense_exploded for FBgn and FBtr that exist in df_Dmel_recap
df_DEST_missense = df_DEST_missense[
    (df_DEST_missense['FBgn_ID'].isin(df_Dmel_recap_pp['FBgn_ID'])) &
    (df_DEST_missense['FBtr_ID'].isin(df_Dmel_recap_pp['FBtr_ID']))
]
# Display the filtered DataFrame
print('filter FBgn and FBtr', df_DEST_missense.shape[0], 'SNPs')
# Merge the filtered DataFrame with df_Dmel_recap_pp on FBtr_ID
df_DEST_missense = df_DEST_missense.merge(df_Dmel_recap_pp[['FBtr_ID', 'Length', 'FBpp_ID']], on='FBtr_ID', how='left')


filter FBgn and FBtr 773511 SNPs


In [ ]:
# Verify if the Protein_length corresponds to Length
df_DEST_missense['Length_match'] = df_DEST_missense['Length_x'].astype(int) == df_DEST_missense['Length_y']

# Display the rows where the lengths do not match
length_mismatch = df_DEST_missense[~df_DEST_missense['Length_match']]
print(f'Total mismatches: {length_mismatch.shape[0]}')
df_DEST_missense = df_DEST_missense[(df_DEST_missense['Length_match'])&(df_DEST_missense['FBpp_ID'].isin(df_Dmel_recap.loc[df_Dmel_recap['Representative_FBpp']==True].index.tolist()))].copy()
print(f'Dataframe with representative proteoforms with matching length: {df_DEST_missense.shape[0]}')


df_DEST_missense['WT_AA_match'] = df_DEST_missense.apply(check_wt_aa, axis=1)
df_DEST_missense = df_DEST_missense.loc[df_DEST_missense['WT_AA_match']]
print(f'Dataframe with representative proteoforms with matching length and wild-type amino acid: {df_DEST_missense.shape[0]}')#539383
# Merge df_snp_representative with df_Dmel_recap to get F_obs values
df_snp_representative = df_DEST_missense.merge(df_Dmel_recap_pp[['FBpp_ID','GEMME_GlobalConfidence', 'F_obs']], on='FBpp_ID', how='left')
# create a dico with the proteocast data (~2 min for 20k proteins)
gemme_data = create_gemme_dico(df_snp_representative)
## remove duplicates C>A,T producing the same aa change for example. 
df_snp_representative = df_snp_representative.loc[~df_snp_representative.duplicated(subset=['FBgn_ID','FBpp_ID', 'Mutation'], keep='first')]
print('without redundant aa changes', df_snp_representative.shape)
## add GEMME scores and rank, variant class and local confidence (~15 minutes for 20k proteins)
df_snp_representative[['GEMME_score','GEMME_rank', 'Variant_class', 'GEMME_LocalConfidence']] = df_snp_representative.apply(lambda row: pd.Series(get_gemme_score_class(row, gemme_data)), axis=1)

## create a SNP_ID by combining the chromosome and the position
df_snp_representative['SNP_ID'] = df_snp_representative['#CHROM'].astype(str) + '_' + df_snp_representative['POS'].astype(str)
## keep only frequencies and depth information from INFO column
df_snp_representative['INFO'] = df_snp_representative['INFO'].apply(lambda x: x.split('ANN')[0])

df_snp_representative = df_snp_representative[['SNP_ID', 'INFO', 'FBgn_ID', 'FBtr_ID', 'Mutation', 'Residue',
       'Length_x', 'FBpp_ID', 'GEMME_GlobalConfidence', 'GEMME_score', 'GEMME_rank', 'Variant_class',
       'GEMME_LocalConfidence', 'F_obs']].copy()
df_snp_representative.rename(columns={'Length_x':'Length'}, inplace=True)
df_snp_representative['Set_name'] = 'DEST2'

Total mismatches: 0
Dataframe with representative proteoforms with matching length: 600315
Dataframe with representative proteoforms with matching length and wild-type amino acid: 600315
without redundant aa changes (599153, 22)


In [39]:
df_snp_representative.to_csv(f'{WORK_DIR}/csv/DEST_allRepresentativeProteoforms.csv', index=False)

### keep only the proteoform with max F_obs per SNP

In [48]:
## remove rows with nan values meaning we don't have GEMME predictions for these proteins
df_snp_representative = df_snp_representative.loc[~df_snp_representative['F_obs'].isnull()]
# Group by unique SNPs and select the row with the maximum F_obs for each group
df_snp_max_fobs = df_snp_representative.loc[df_snp_representative.groupby(['SNP_ID','FBgn_ID'])['F_obs'].idxmax()]
# Display the result
print(f'Number of unique SNPs: {df_snp_max_fobs.shape[0]}')
df_snp_max_fobs.head()

Number of unique SNPs: 331362


,SNP_ID,INFO,FBgn_ID,FBtr_ID,Mutation,Residue,Length,FBpp_ID,GEMME_GlobalConfidence,GEMME_score,GEMME_rank,Variant_class,GEMME_LocalConfidence,F_obs,Set_name
77888,2L_10000016,ADP=51.2535;DP=36800;NC=35;AF=0.544433;AC=20141;,FBgn0051755,FBtr0303882,E471D,471,1476,FBpp0292885,True,-2.133489,0.531,neutral,True,29.24,DEST2
77889,2L_10000023,ADP=52.3055;DP=36980;NC=46;AF=0.00352139;AC=176;,FBgn0051755,FBtr0303882,S469T,469,1476,FBpp0292885,True,-3.987892,0.834,impactful,True,29.24,DEST2
77890,2L_10000029,ADP=51.5216;DP=36941;NC=36;AF=0.00914833;AC=318;,FBgn0051755,FBtr0303882,A467G,467,1476,FBpp0292885,True,-4.775772,0.910,impactful,True,29.24,DEST2
77891,2L_10000059,ADP=51.8454;DP=36551;NC=48;AF=0.0129318;AC=551;,FBgn0051755,FBtr0303882,P457L,457,1476,FBpp0292885,True,-0.703098,0.222,neutral,True,29.24,DEST2
77892,2L_10000089,ADP=51.5622;DP=35217;NC=70;AF=0.612867;AC=21823;,FBgn0051755,FBtr0303882,C447Y,447,1476,FBpp0292885,True,-0.198017,0.088,neutral,True,29.24,DEST2


In [63]:
## add GMM3zscore
df_snp_max_fobs[ 'GMM3_zscoreNeutral'] = df_snp_max_fobs.apply(lambda row: pd.Series(calculate_gmm3_class_and_zscore(row, gemme_data)), axis=1)

In [94]:
df_snp_max_fobs.to_csv(f'{WORK_DIR}/csv/DEST_maxFobsProteoforms.csv', index=False)

In [67]:
del gemme_data

## Hypomorphic data 

In [68]:
## Filter raws without mapping to the genome information
df_HypomorphicRaw = df_HypomorphicRaw[df_HypomorphicRaw['MUTATIONS_MAPPED_TO_GENOME']!='-']
df_HypomorphicRaw.rename(columns={'#SUBMITTED ID':'Allele'}, inplace=True)

def extract_missense_variants_Hypomorphic(row):

    phenotypic_class = row['PHENOTYPIC_CLASS'].split('<newline>')
    phenotypes = sum([patternPhenotype.findall(x) for x in phenotypic_class], [])

    mutations_mapped = patternMutProt.findall(row['MUTATIONS_MAPPED_TO_GENOME'])
    l_mut=[]; l_prot=[]
    if mutations_mapped:
        l_mut_prot = [(i.split(' | ')[0],i.split(' | ')[1])  for i in mutations_mapped]
        for mut, prot in l_mut_prot:
            prot=CorrectProtname(prot)
            try:
                FBpp_ID = df_Dmel_recap.loc[df_Dmel_recap['Protein_symbol']==prot].index[0]
                if FBpp_ID:
                    l_mut.append(mut)
                    l_prot.append(FBpp_ID)
                else:
                    print('protein symbol not found:', prot)
            except:
                print('protein symbol not found:', prot)
                continue

        return l_mut, l_prot, phenotypes
    else:
        return None, None, None
            
        


In [104]:
df_Hypomorphic = df_HypomorphicRaw.copy()
df_Hypomorphic[['Mutation', 'FBpp_ID', 'Phenotype']] = df_Hypomorphic.apply(lambda row: pd.Series(extract_missense_variants_Hypomorphic(row)), axis=1)
df_Hypomorphic=df_Hypomorphic[['Allele', 'Mutation', 'FBpp_ID', 'Phenotype']].copy()
# Explode the lists in the DataFrame
df_Hypomorphic = df_Hypomorphic.explode(['FBpp_ID','Mutation'])
df_Hypomorphic = df_Hypomorphic.loc[~df_Hypomorphic['FBpp_ID'].isnull()].copy()
# Reset the index after exploding
df_Hypomorphic.reset_index(drop=True, inplace=True)
df_Hypomorphic = df_Hypomorphic.loc[(df_Hypomorphic['FBpp_ID'].isin(df_Dmel_recap_pp['FBpp_ID']))]
df_Hypomorphic['Set_name'] = 'Hypomorphic'

#### verify
df_Hypomorphic = df_Hypomorphic.merge(df_Dmel_recap_pp[['FBtr_ID','FBgn_ID', 'GEMME_GlobalConfidence', 'F_obs', 'Length', 'FBpp_ID']], on='FBpp_ID', how='left')

df_Hypomorphic = df_Hypomorphic[(df_Hypomorphic['FBpp_ID'].isin(df_Dmel_recap.loc[df_Dmel_recap['Representative_FBpp']==True].index.tolist()))].copy()
print(f'Dataframe with representative proteoforms with matching length: {df_Hypomorphic.shape[0]}')
df_Hypomorphic['Residue'] = df_Hypomorphic['Mutation'].apply(lambda x: int(x[1:-1]))
df_Hypomorphic['WT_AA_match'] = df_Hypomorphic.apply(check_wt_aa, axis=1)
df_Hypomorphic = df_Hypomorphic.loc[df_Hypomorphic['WT_AA_match']]
print(f'Dataframe with representative proteoforms with matching length and wild-type amino acid: {df_Hypomorphic.shape[0]}')
# create a dico with the proteocast data 
gemme_data = create_gemme_dico(df_Hypomorphic)
## remove duplicates C>A,T producing the same aa change for example. 
df_Hypomorphic = df_Hypomorphic.loc[~df_Hypomorphic.duplicated(subset=['FBgn_ID','FBpp_ID', 'Mutation'], keep='first')].copy()
print('without redundant aa changes', df_Hypomorphic.shape)
## add GEMME scores and rank, variant class and local confidence
df_Hypomorphic[['GEMME_score','GEMME_rank', 'Variant_class', 'GEMME_LocalConfidence']] = df_Hypomorphic.apply(lambda row: pd.Series(get_gemme_score_class(row, gemme_data)), axis=1)
del df_Hypomorphic['WT_AA_match']

protein symbol not found: bru1-PL
Dataframe with representative proteoforms with matching length: 950
Dataframe with representative proteoforms with matching length and wild-type amino acid: 950
without redundant aa changes (928, 12)


In [105]:
df_Hypomorphic.to_csv(f'{WORK_DIR}/csv/Hypomorphic_allRepresentativeProteoforms.csv', index=False)

### Keep only one FBpp per SNP

In [108]:
print(df_Hypomorphic.shape)
## remove rows with nan values meaning we don't have GEMME predictions for these proteins
df_Hypomorphic = df_Hypomorphic.loc[~df_Hypomorphic['F_obs'].isnull()]
print(df_Hypomorphic.shape)
# Group by unique SNPs and select the row with the maximum F_obs for each group
df_Hypomorphic_max_fobs = df_Hypomorphic.loc[df_Hypomorphic.groupby(['Allele', 'FBgn_ID'])['F_obs'].idxmax()]
# Display the result
print(f'Number of unique SNPs: {df_Hypomorphic_max_fobs.shape[0]}')
## add GMM3zscore (takes ~10 minutes for  proteins)
df_Hypomorphic_max_fobs[ 'GMM3_zscoreNeutral'] = df_Hypomorphic_max_fobs.apply(lambda row: pd.Series(calculate_gmm3_class_and_zscore(row, gemme_data)), axis=1)
df_Hypomorphic_max_fobs['is_lethal'] = df_Hypomorphic_max_fobs['Phenotype'].apply(lambda x: any(('lethal' in phenotype) and ('partially' not in phenotype) for phenotype in x) if isinstance(x, list) else False)

(928, 16)
(928, 16)
Number of unique SNPs: 403


,Allele,Mutation,FBpp_ID,Phenotype,Set_name,FBtr_ID,FBgn_ID,GEMME_GlobalConfidence,F_obs,Length,Residue,WT_AA_match,GEMME_score,GEMME_rank,Variant_class,GEMME_LocalConfidence
170,FBal0000017,P348S,FBpp0081153,"[lethal, recessive, male semi-sterile]",Hypomorphic,FBtr0081639,FBgn0003884,True,96.84,450,348,True,-1.839196,0.534,uncertain,True
1076,FBal0000019,V177M,FBpp0081153,"[female semi-sterile, viable, male sterile, re...",Hypomorphic,FBtr0081639,FBgn0003884,True,96.84,450,177,True,-0.224037,0.121,neutral,True
608,FBal0000027,G56D,FBpp0081524,"[electrophoretic variant, male sterile, male f...",Hypomorphic,FBtr0082046,FBgn0003889,True,96.78,446,56,True,-0.092218,0.080,neutral,True
1415,FBal0000028,D114N,FBpp0081524,"[electrophoretic variant, male sterile, abnorm...",Hypomorphic,FBtr0082046,FBgn0003889,True,96.78,446,114,True,-0.704746,0.254,neutral,True
1116,FBal0000267,R29C,FBpp0082597,"[flightless, dominant]",Hypomorphic,FBtr0083143,FBgn0000047,True,98.19,376,29,True,-0.662521,0.253,neutral,True


In [ ]:
df_Hypomorphic_max_fobs.to_csv(f'{WORK_DIR}/csv/Hypomorphic_maxFobsProteoforms.csv', index=False)

In [111]:
del gemme_data

## DGRP

In [24]:
def extract_missense_variants_dgrp(row):
    
    l_FBtr=[]; l_Mut = [];l_prot_len = []
   
    tr_annot = (re.search('(?<=TranscriptAnnot\[).*(?=\])', row['GeneAnnotation'])).group(0).split(';')                                 ## transcripts and mutations (#mut = #SNPs)
    tr_annot = [
        (patternTrans.search(x).group(0), patternMut.search(x).group(0), x.split('|')[4])
        for x in tr_annot if 'NON_SYNONYMOUS_CODING' in x and 'MISSENSE' in x
    ]

    l_FBtr, l_Mut, l_prot_len = zip(*tr_annot) if tr_annot else ([], [], [])

    
    return l_FBtr, l_Mut, l_prot_len

In [39]:
df_DGRPRaw_missense = df_DGRPRaw[df_DGRPRaw['GeneAnnotation'].str.contains('NON_SYNONYMOUS_CODING')]
del df_DGRPRaw
del df_DGRPRaw_missense['RegulationAnnotation']

In [44]:
"""# Apply the function to the DataFrame
df_DGRPRaw_missense[['FBtr_ID', 'Mutation', 'Length']] = df_DGRPRaw_missense.apply(lambda x: pd.Series(extract_missense_variants_dgrp(x)), axis=1)
# Explode the lists in the DataFrame
df_DGRPRaw_missense = df_DGRPRaw_missense.explode(['FBtr_ID', 'Mutation', 'Length'])"""
print(df_DGRPRaw_missense.shape)
# Filter FBtr that exist in df_Dmel_recap
df_DGRPRaw_missense = df_DGRPRaw_missense[
    (df_DGRPRaw_missense['FBtr_ID'].isin(df_Dmel_recap_pp['FBtr_ID']))
]
# Display the filtered DataFrame
print('filter FBgn and FBtr', df_DGRPRaw_missense.shape[0], 'SNPs')
# Merge the filtered DataFrame with df_Dmel_recap_pp on FBtr_ID
df_DGRPRaw_missense = df_DGRPRaw_missense.merge(df_Dmel_recap_pp[['FBgn_ID', 'FBtr_ID', 'Length', 'FBpp_ID', 'GEMME_GlobalConfidence', 'F_obs']], on='FBtr_ID', how='left', suffixes=('_x', '_y'))

# Verify if the Protein_length corresponds to Length
df_DGRPRaw_missense['Length_match'] = df_DGRPRaw_missense['Length_x'].astype(int) == df_DGRPRaw_missense['Length_y']
df_DGRPRaw_missense = df_DGRPRaw_missense[(df_DGRPRaw_missense['Length_match'])&(df_DGRPRaw_missense['FBpp_ID'].isin(df_Dmel_recap.loc[df_Dmel_recap['Representative_FBpp']==True].index.tolist()))].copy()
print(f'Dataframe with representative proteoforms with matching length: {df_DGRPRaw_missense.shape[0]}')

df_DGRPRaw_missense['Residue'] = df_DGRPRaw_missense['Mutation'].apply(lambda x: int(x[1:-1]))
## check if the wild-type amino acid in the annotations matches the one in the protein sequence
df_DGRPRaw_missense['WT_AA_match'] = df_DGRPRaw_missense.apply(check_wt_aa, axis=1)
df_DGRPRaw_missense = df_DGRPRaw_missense.loc[df_DGRPRaw_missense['WT_AA_match']]
print(f'Dataframe with representative proteoforms with matching length and wild-type amino acid: {df_DGRPRaw_missense.shape[0]}')

# Start the timer
start_time = time.time()
# create a dico with the proteocast data 
gemme_data = create_gemme_dico(df_DGRPRaw_missense)
end_time = time.time()
# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"gemme dico took: {elapsed_time} seconds for {len(gemme_data)} proteins")


## remove duplicates C>A,T producing the same aa change for example. 
df_DGRPRaw_missense = df_DGRPRaw_missense.loc[~df_DGRPRaw_missense.duplicated(subset=['FBgn_ID','FBpp_ID', 'Mutation'], keep='first')]
print('without redundant aa changes', df_DGRPRaw_missense.shape)

# Start the timer
start_time = time.time()
## add GEMME scores and rank, variant class and local confidence (~15 minutes for 20k proteins)
df_DGRPRaw_missense[['GEMME_score','GEMME_rank', 'Variant_class', 'GEMME_LocalConfidence']] = df_DGRPRaw_missense.apply(lambda row: pd.Series(get_gemme_score_class(row, gemme_data)), axis=1)
end_time = time.time()
# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"add gemme scores etc took: {elapsed_time} seconds")
df_DGRPRaw_missense = df_DGRPRaw_missense[['ID', 'FBgn_ID', 'FBtr_ID', 'FBpp_ID', 'Mutation', 'Residue',
       'Length_x', 'GEMME_GlobalConfidence', 'GEMME_score', 'GEMME_rank', 'Variant_class',
       'GEMME_LocalConfidence', 'F_obs']].copy()
df_DGRPRaw_missense.rename(columns={'Length_x':'Length', 'ID':'SNP_ID'}, inplace=True)
df_DGRPRaw_missense['Set_name'] = 'DGRP'
df_DGRPRaw_missense.to_csv(f'{WORK_DIR}/csv/DGRP_allRepresentativeProteoforms.csv', index=False)

(399149, 6)
filter FBgn and FBtr 391515 SNPs
Dataframe with representative proteoforms with matching length: 308361
Dataframe with representative proteoforms with matching length and wild-type amino acid: 308092
gemme dico took: 184.2169952392578 seconds for 20009 proteins
without redundant aa changes (308050, 14)
add gemme scores etc took: 487.3049352169037 seconds
Index(['ID', 'allele', 'GeneAnnotation', 'FBtr_ID', 'Mutation', 'Length_x',
       'FBgn_ID', 'Length_y', 'FBpp_ID', 'GEMME_GlobalConfidence', 'F_obs',
       'Length_match', 'Residue', 'WT_AA_match', 'GEMME_score', 'GEMME_rank',
       'Variant_class', 'GEMME_LocalConfidence'],
      dtype='object')


### Keep only one proteoform with max F_obs per SNP

In [62]:
print(df_DGRPRaw_missense.shape)
## remove rows with nan values meaning we don't have GEMME predictions for these proteins
df_DGRPRaw_missense = df_DGRPRaw_missense.loc[~df_DGRPRaw_missense['F_obs'].isnull()]
print(df_DGRPRaw_missense.shape)
# Group by unique SNPs and select the row with the maximum F_obs for each group
df_DGRPRaw_missense_max_fobs = df_DGRPRaw_missense.loc[df_DGRPRaw_missense.groupby(['SNP_ID', 'FBgn_ID'])['F_obs'].idxmax()]
# Display the result
print(f'Number of unique SNPs: {df_DGRPRaw_missense_max_fobs.shape[0]}')
## add GMM3zscore (takes ~10 minutes for  proteins)
df_DGRPRaw_missense_max_fobs[ 'GMM3_zscoreNeutral'] = df_DGRPRaw_missense_max_fobs.apply(lambda row: pd.Series(calculate_gmm3_class_and_zscore(row, gemme_data)), axis=1)

(308050, 14)
(306674, 14)
Number of unique SNPs: 177013


In [63]:
df_DGRPRaw_missense_max_fobs.to_csv(f'{WORK_DIR}/csv/DGRP_maxFobsProteoforms.csv', index=False)

In [64]:
del gemme_data

### Frequencies of the SNPs

- 205 inbred lines
- cut -d $' ' -f10- dgrp2.tgeno.txt > dgrp2.lines.homozygote
- grep -n '1' dgpr2.lines.homozygote -------------> 0 SNPs found, all homozygotes

In [73]:
df_DGRP_SNPfreq = pd.read_csv(f'{WORK_DIR}/data/dgrp2.tgeno.txt', sep=' ')
print(df_DGRP_SNPfreq.shape)
## remove lines' information columns
df_DGRP_SNPfreq = df_DGRP_SNPfreq[['id','ref', 'alt', 'refc', 'altc', 'qual', 'cov']].copy()
df_DGRP_SNPfreq = df_DGRP_SNPfreq.loc[df_DGRP_SNPfreq['id'].str.contains('SNP')].copy()
print(df_DGRP_SNPfreq.shape, type(df_DGRP_SNPfreq.loc[0, 'refc']))

#df_DGRP_SNPfreq['altc'] = df_DGRP_SNPfreq['altc'].astype('int')
#genotypes['refc'] = genotypes['refc'].astype('int')
df_DGRP_SNPfreq['countAllele'] = df_DGRP_SNPfreq['altc']+df_DGRP_SNPfreq['refc']
df_DGRP_SNPfreq['MAF_A'] = df_DGRP_SNPfreq['altc']/df_DGRP_SNPfreq['countAllele'] ## alternative allele frequency 
df_DGRP_SNPfreq['MAF_R'] = df_DGRP_SNPfreq['refc']/df_DGRP_SNPfreq['countAllele'] ## reference allele frequency 

/var/folders/v9/tv7qw0pd675c44_m3d98bz_c0000gn/T/ipykernel_20521/86042631.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_DGRP_SNPfreq = pd.read_csv(f'{WORK_DIR}/data/dgrp2.tgeno.txt', sep=' ')


(4438427, 214)
(3963420, 7) <class 'numpy.int64'>


In [86]:
df_DGRP_SNPfreq = df_DGRP_SNPfreq.loc[df_DGRP_SNPfreq['id'].isin(df_DGRPRaw_missense_max_fobs['SNP_ID'].unique())].copy()

In [88]:
df_DGRP_SNPfreq ## ref and alt: ref is the Allele in DGRPRaw file, but the codon change might different due to the strand orientation.

,id,ref,alt,refc,altc,qual,cov,countAllele,MAF_A,MAF_R
117,2L_11255_SNP,A,T,197,1,999,26,198,0.005051,0.994949
126,2L_12460_SNP,T,C,133,61,999,40,194,0.314433,0.685567
127,2L_12472_SNP,G,T,204,1,999,39,205,0.004878,0.995122
128,2L_12486_SNP,A,C,201,4,999,39,205,0.019512,0.980488
139,2L_13859_SNP,A,T,203,2,999,36,205,0.009756,0.990244
...,...,...,...,...,...,...,...,...,...,...
4438245,X_22395718_SNP,G,T,185,20,999,25,205,0.097561,0.902439
4438246,X_22395794_SNP,G,A,184,16,999,21,200,0.080000,0.920000
4438254,X_22396342_SNP,T,G,121,80,999,21,201,0.398010,0.601990
4438256,X_22397129_SNP,C,T,200,2,999,18,202,0.009901,0.990099


In [95]:
df_DGRP_SNPfreq.to_csv(f'{WORK_DIR}/csv/DGRP_SNPfreq.csv', index=False)

 - 2 810 208 1% 
 - 1 796 221 5%

## Lethal

In [312]:
## Filter raws without mapping to the genome information
df_LethalRaw = pd.read_csv(f'{WORK_DIR}/data/LethalAllele_EMS.txt', sep='\t')
df_LethalRaw = df_LethalRaw[df_LethalRaw['MUTATIONS_MAPPED_TO_GENOME']!='-']
df_LethalRaw.rename(columns={'#SUBMITTED ID':'Allele'}, inplace=True)

def extract_missense_variants_Lethal(row):
    l_mut=[]; l_prot=[];l_nachange=[]
    for x in row['MUTATIONS_MAPPED_TO_GENOME'].split('<newline>'):
        mutations_mapped = patternMutProt.findall(x)
        
        
        if mutations_mapped:
            l_mut_prot = [(i.split(' | ')[0],i.split(' | ')[1])  for i in mutations_mapped]
            na_change = patternNAchange.findall(x)[0]                      ## associated Nucleotide changes to the allele
            for mut, prot in l_mut_prot:
                prot=CorrectProtname(prot)
                try:
                    FBpp_ID = df_Dmel_recap.loc[df_Dmel_recap['Protein_symbol']==prot].index[0]
                    if FBpp_ID:
                        l_mut.append(mut)
                        l_prot.append(FBpp_ID)
                        l_nachange.append(na_change)
                    else:
                        print('protein symbol not found:', prot)
                except:
                    print('protein symbol not found:', prot)
                    continue
    return l_nachange, l_mut, l_prot 


In [313]:
df_LethalRaw = df_LethalRaw.copy()
df_LethalRaw[['SNP_ID', 'Mutation', 'FBpp_ID']] = df_LethalRaw.apply(lambda row: pd.Series(extract_missense_variants_Lethal(row)), axis=1)
# Explode the lists in the DataFrame
df_LethalRaw = df_LethalRaw.explode(['SNP_ID', 'FBpp_ID','Mutation'])
df_LethalRaw = df_LethalRaw.loc[~df_LethalRaw['FBpp_ID'].isnull()].copy()
# Reset the index after exploding
df_LethalRaw.reset_index(drop=True, inplace=True)
df_LethalRaw = df_LethalRaw.loc[(df_LethalRaw['FBpp_ID'].isin(df_Dmel_recap_pp['FBpp_ID']))]
df_LethalRaw['Set_name'] = 'Lethal'
#### verify
df_LethalRaw = df_LethalRaw.merge(df_Dmel_recap_pp[['FBtr_ID', 'FBgn_ID', 'GEMME_GlobalConfidence', 'F_obs', 'Length', 'FBpp_ID']], on='FBpp_ID', how='left')

df_LethalRaw = df_LethalRaw[(df_LethalRaw['FBpp_ID'].isin(df_Dmel_recap.loc[df_Dmel_recap['Representative_FBpp']==True].index.tolist()))].copy()
print(f'Dataframe with representative proteoforms with matching length: {df_LethalRaw.shape[0]}')
df_LethalRaw['Residue'] = df_LethalRaw['Mutation'].apply(lambda x: int(x[1:-1]))
df_LethalRaw['WT_AA_match'] = df_LethalRaw.apply(check_wt_aa, axis=1)
df_LethalRaw = df_LethalRaw.loc[df_LethalRaw['WT_AA_match']]
print(f'Dataframe with representative proteoforms with matching length and wild-type amino acid: {df_LethalRaw.shape[0]}')
# create a dico with the proteocast data 
gemme_data = create_gemme_dico(df_LethalRaw)
## remove duplicates C>A,T producing the same aa change for example. 
df_LethalRaw = df_LethalRaw.loc[~df_LethalRaw.duplicated(subset=['FBgn_ID','FBpp_ID', 'Mutation'], keep='first')].copy()
print('without redundant aa changes', df_LethalRaw.shape)
## add GEMME scores and rank, variant class and local confidence
df_LethalRaw[['GEMME_score','GEMME_rank', 'Variant_class', 'GEMME_LocalConfidence']] = df_LethalRaw.apply(lambda row: pd.Series(get_gemme_score_class(row, gemme_data)), axis=1)
## in some annotations we find comments like 'Two adjacent amino acids...' that we need to remove
df_LethalRaw = df_LethalRaw.loc[~df_LethalRaw['MUTATIONS_MAPPED_TO_GENOME'].str.contains('Two')].copy()
##manual correction
df_LethalRaw = df_LethalRaw.loc[df_LethalRaw['Allele']!='FBal0230457']
#Src42A-PC doesn't exist in 6.44 version
df_LethalRaw = df_LethalRaw[['Allele','MUTATIONS_MAPPED_TO_GENOME', 'SNP_ID', 'Mutation', 'FBpp_ID', 'Set_name', 'FBtr_ID',
       'FBgn_ID', 'GEMME_GlobalConfidence', 'F_obs', 'Length', 'Residue',
       'GEMME_score', 'GEMME_rank', 'Variant_class',
       'GEMME_LocalConfidence']].copy()

protein symbol not found: Src42A-PC
protein symbol not found: Src42A-PD
protein symbol not found: Src42A-PC
Dataframe with representative proteoforms with matching length: 2485
Dataframe with representative proteoforms with matching length and wild-type amino acid: 2485
without redundant aa changes (2365, 18)


In [255]:
df_LethalRaw.to_csv(f'{WORK_DIR}/csv/Lethal_allRepresentativeProteoforms.csv', index=False)

### Keep only one FBpp per SNP

In [288]:
print(df_LethalRaw.shape)
## remove rows with nan values meaning we don't have GEMME predictions for these proteins
df_LethalRaw = df_LethalRaw.loc[~df_LethalRaw['F_obs'].isnull()]
print(df_LethalRaw.shape)
# Group by unique SNPs and select the row with the maximum F_obs for each group
df_LethalRaw_max_fobs = df_LethalRaw.loc[df_LethalRaw.groupby(['Allele', 'SNP_ID', 'FBgn_ID'])['F_obs'].idxmax()]

## comment=Nucleotide substitution reported with respect to ewg-PE isoform. na_change=C271604A pr_change=Q544K | ewg-PA; Q544K | ewg-PB; Q544K | ewg-PC; R568R | ewg-PD; Q544K | ewg-PE; Q390K | ewg-PF; Q390K | ewg-PG; R414R | ewg-PH; Q393K | ewg-PJ; Q547K | ewg-PK; Q532K | ewg-PL; Q499K | ewg-PM
## and in the annotations the proteoform with the maximum Fobs which is ewg-PD keeps the WT R568R
df_LethalRaw_max_fobs.loc[2075] = df_LethalRaw.loc[2076]

# Display the result
print(f'Number of unique SNPs: {df_LethalRaw_max_fobs.shape[0]}')
## add GMM3zscore (takes ~10 minutes for  proteins)
df_LethalRaw_max_fobs[ 'GMM3_zscoreNeutral'] = df_LethalRaw_max_fobs.apply(lambda row: pd.Series(calculate_gmm3_class_and_zscore(row, gemme_data)), axis=1)

(2348, 16)
(2348, 16)
Number of unique SNPs: 1066


In [289]:
df_LethalRaw_max_fobs.to_csv(f'{WORK_DIR}/csv/Lethal_maxFobsProteoforms.csv', index=False)

## ALL 

if GEMME_GlobalConfidence is NaN it means we don't have the predictions

In [39]:
df_DEST = pd.read_csv(f'{WORK_DIR}/csv/DEST_maxFobsProteoforms.csv', sep=',')
del df_DEST['INFO']
df_Hypomorphic = pd.read_csv(f'{WORK_DIR}/csv/Hypomorphic_maxFobsProteoforms.csv', sep=',')
df_Hypomorphic['Phenotype'] = df_Hypomorphic['Phenotype'].apply(eval)
del df_Hypomorphic['Phenotype']
del df_Hypomorphic['is_lethal']
df_DGRP = pd.read_csv(f'{WORK_DIR}/csv/DGRP_maxFobsProteoforms.csv', sep=',')
df_Lethal = pd.read_csv(f'{WORK_DIR}/csv/Lethal_maxFobsProteoforms.csv', sep=',')
del df_Lethal['MUTATIONS_MAPPED_TO_GENOME']

df_ProteoCast_Benchmark = pd.concat([df_DEST, df_DGRP, df_Hypomorphic, df_Lethal], axis=0, ignore_index=True)
#del df_ProteoCast_Benchmark['Allele']

In [40]:
tmp = df_ProteoCast_Benchmark.loc[(df_ProteoCast_Benchmark['Set_name'].isin(["DEST2", 'Lethal']))].copy()
ind_lethal_DEST = tmp[(tmp.duplicated(subset=['FBpp_ID', 'Mutation'], keep=False))].index.tolist()
tmp = df_ProteoCast_Benchmark.loc[(df_ProteoCast_Benchmark['Set_name'].isin(["DGRP", 'Lethal']))].copy()
ind_lethal_DGRP = tmp[(tmp.duplicated(subset=['FBpp_ID', 'Mutation'], keep=False))].index.tolist()

ind_lethal_DEST_DGRP = set(ind_lethal_DGRP).intersection(set(ind_lethal_DEST))
ind_leth_DEST_DGRP_all  = set(ind_lethal_DGRP)|set(ind_lethal_DEST)
df_ProteoCast_Benchmark.loc[(df_ProteoCast_Benchmark.index.isin(list(ind_leth_DEST_DGRP_all)))&(df_ProteoCast_Benchmark['Set_name']=='Lethal')].to_csv('../csv/ProteoCast_Benchmark_Lethal_DEST2_DGRP.csv', index=False)

In [321]:
df_ProteoCast_Benchmark.loc[df_ProteoCast_Benchmark['Set_name'].isin(['DGRP', 'DEST2']),'True_label']=0
df_ProteoCast_Benchmark.loc[df_ProteoCast_Benchmark['Set_name']=='Lethal','True_label']=1

df_ProteoCast_Benchmark['Variant_label'] = df_ProteoCast_Benchmark['Variant_class']
df_ProteoCast_Benchmark.loc[:, 'Variant_class'] = df_ProteoCast_Benchmark['Variant_class'].replace({'neutral': 0, 'uncertain': 0.5, 'impactful': 1})

In [315]:
#dico_sameMut = {} no need anymore
for set_name in ['Lethal', 'Hypomorphic', 'DGRP', 'DEST2', ]:
    same_mut = df_ProteoCast_Benchmark.loc[df_ProteoCast_Benchmark['Set_name']==set_name].copy()
    print(set_name, same_mut.shape)
    #dico_sameMut[set_name] = same_mut.loc[same_mut.duplicated(subset=['FBpp_ID', 'Mutation'], keep='first')].index.tolist()
#combined_list = [item for sublist in dico_sameMut.values() for item in sublist]
#df_ProteoCast_Benchmark = df_ProteoCast_Benchmark.loc[~df_ProteoCast_Benchmark.index.isin(combined_list)].copy()

Lethal (1066, 16)
Hypomorphic (403, 16)
DGRP (177013, 16)
DEST2 (331341, 16)


In [322]:
df_ProteoCast_Benchmark.to_csv(f'{WORK_DIR}/csv/ProteoCast_Benchmark.csv', index=False)

## Add number of mutations to the recapitulative file

In [28]:
df_DEST = pd.read_csv(f'{WORK_DIR}/csv/DEST_allRepresentativeProteoforms.csv', sep=',')
del df_DEST['INFO']
df_Hypomorphic = pd.read_csv(f'{WORK_DIR}/csv/Hypomorphic_allRepresentativeProteoforms.csv', sep=',')
df_Hypomorphic['Phenotype'] = df_Hypomorphic['Phenotype'].apply(eval)
del df_Hypomorphic['Phenotype']
df_DGRP = pd.read_csv(f'{WORK_DIR}/csv/DGRP_allRepresentativeProteoforms.csv', sep=',')
df_Lethal = pd.read_csv(f'{WORK_DIR}/csv/Lethal_allRepresentativeProteoforms.csv', sep=',')

In [ ]:
df_DEST['FBpp_ID'].nunique(), df_DEST['FBgn_ID'].nunique() 

(20694, 13110)

In [8]:
df_natPop = pd.concat([df_DEST, df_DGRP], axis=0, ignore_index=True)
common_mut_ind = df_natPop.loc[df_natPop.duplicated(subset=['FBpp_ID', 'Mutation'], keep=False)].index.tolist()#['Set_name'].value_counts()#.sort_values(by=['FBpp_ID', 'Mutation'])
union_mut_ind = df_natPop.loc[(df_natPop.duplicated(subset=['FBpp_ID', 'Mutation'], keep=False))&(df_natPop['Set_name']=='DEST2')].index.tolist()#['Set_name'].value_counts()#.sort_values(by=['FBpp_ID', 'Mutation'])
df_natPop_unique = df_natPop.loc[~df_natPop.index.isin(common_mut_ind)].copy()
df_natPop_union = df_natPop.drop(index=union_mut_ind).copy()

In [41]:
# Create a dictionary to store the counts and unique residues
counts = {
    'n_Lethal': df_Lethal['FBpp_ID'].value_counts(),
    'n_Lethal_res': df_Lethal.groupby('FBpp_ID')['Residue'].nunique(),
    'n_Hypomorphic': df_Hypomorphic['FBpp_ID'].value_counts(),
    'n_Hypomorphic_res': df_Hypomorphic.groupby('FBpp_ID')['Residue'].nunique(),
    'n_DEST2': df_natPop_unique[df_natPop_unique['Set_name'] == 'DEST2']['FBpp_ID'].value_counts(),
    'n_DEST2_res': df_natPop_unique[df_natPop_unique['Set_name'] == 'DEST2'].groupby('FBpp_ID')['Residue'].nunique(),
    'n_DGRP': df_natPop_unique[df_natPop_unique['Set_name'] == 'DGRP']['FBpp_ID'].value_counts(),
    'n_DGRP_res': df_natPop_unique[df_natPop_unique['Set_name'] == 'DGRP'].groupby('FBpp_ID')['Residue'].nunique(),
    'n_DEST_DGRP_union': df_natPop_union['FBpp_ID'].value_counts(),
    'n_DEST_DGRP_union_res': df_natPop_union.groupby('FBpp_ID')['Residue'].nunique()
}

# Assign the counts to the DataFrame
for col, count in counts.items():
    df_Dmel_recap[col] = df_Dmel_recap.index.map(count).fillna(0)

In [60]:
df_Dmel_recap.to_csv(f'{WORK_DIR}/csv/Dmel6.44PredictionsRecap.csv', index=True)